In [7]:
import numpy as np
import os
import re
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd

In [8]:
path = r'C:\Users\Pisces Khan\OneDrive\Documents\Articles'
filenames = os.listdir(path)
print(filenames)

['negative_0.txt', 'negative_1.txt', 'negative_10.txt', 'negative_11.txt', 'negative_12.txt', 'negative_13.txt', 'negative_14.txt', 'negative_15.txt', 'negative_16.txt', 'negative_17.txt', 'negative_18.txt', 'negative_19.txt', 'negative_2.txt', 'negative_20.txt', 'negative_21.txt', 'negative_22.txt', 'negative_23.txt', 'negative_24.txt', 'negative_25.txt', 'negative_26.txt', 'negative_27.txt', 'negative_28.txt', 'negative_29.txt', 'negative_3.txt', 'negative_30.txt', 'negative_31.txt', 'negative_32.txt', 'negative_33.txt', 'negative_4.txt', 'negative_5.txt', 'negative_6.txt', 'negative_7.txt', 'negative_8.txt', 'negative_9.txt', 'neutral_0.txt', 'neutral_1.txt', 'neutral_10.txt', 'neutral_11.txt', 'neutral_12.txt', 'neutral_13.txt', 'neutral_14.txt', 'neutral_15.txt', 'neutral_16.txt', 'neutral_17.txt', 'neutral_18.txt', 'neutral_19.txt', 'neutral_2.txt', 'neutral_20.txt', 'neutral_21.txt', 'neutral_22.txt', 'neutral_23.txt', 'neutral_24.txt', 'neutral_25.txt', 'neutral_26.txt', 'neutr

In [9]:
stopWords = set(stopwords.words('english'))
stopWords.update(['"', "'", ':', '(', ')', '[', ']', '{', '}'])

In [10]:
def getTagsForWords(textLn2):
    tokens=word_tokenize(textLn2)
    tagged=pos_tag(tokens)
    return(tagged)

In [11]:
def remStopWordsOur(lineIn):
    stopWords= {'i','a','and','about','an','are','as','at','be','by','com','for','from','how','in','is','it','not','of','on','or','that','the','this','to','was','what','when','where','who','will','with','the','www','your','is','am','some','you','your','I','A','And','About','An','Are','As','At','Be','By','Com','For','From','How','In','Is','It','Not','Of','On','Or','That','The','This','To','Was','What','When','Where','Who','Will','With','The','Www','Your','Is','Am','Some','You','Your','Was'}
    rmdStopWordsLn = ' '.join(w for w in lineIn.split() if w.lower() not in stopWords)
    return rmdStopWordsLn

In [12]:
def preprocessText(lineIn):
    lineInLower=lineIn.lower()
    lineInRmdSplChars=lineInLower.replace('.',' ').replace(';',' ').replace(',',' ').replace('?',' ').replace('!',' ').replace(':',' ')
    return lineInRmdSplChars

In [13]:
def getAllLines(lineIn):
    lineInReplcByPeriod=lineIn.replace('.','.§').replace(';',';§').replace(',',',§').replace('?','?§').replace('!','!§').replace('\n','§')
    linesOriginal=lineInReplcByPeriod.split('§')
    linesOriginal2=[item for item in linesOriginal if len(item)>0 ]
    return linesOriginal2

In [14]:
def getNounPositions(type,tagged,lineIn):
    nounPosi={}
    for item in tagged:
        if item[1]==type and item[0]!='[' and item[0]!=']':
            nounPosi[item[0]]=-1
    
    for key in nounPosi.keys():
        print('key=',key)
        regExpression=r'\b'+key.lower()+r'\b'
        nounsi=[m.start() for m in re.finditer(regExpression, lineIn.lower())]
        nounPosi[key]=nounsi
    return nounPosi

In [15]:
def getProNounPositions(tagged,lineIn):
    proNounPosi={}
    for item in tagged:
        if item[1]=='PRP': 
            proNounPosi[item[0].lower()]=-1
    
    for key in proNounPosi.keys():
        regExpression=r'\b'+key.lower()+r'\b'
        pronounsi=[m.start() for m in re.finditer(regExpression, lineIn.lower())]
        proNounPosi[key]=pronounsi
    return proNounPosi

In [16]:
def getNearestPreviousNoun(NNP,posiOfPronoun,lineIn):
#    print('\t',NNP)    
    minimumDiff=len(lineIn)
    nearKey=''
    for keyNNP in NNP.keys():
        for posNoun in NNP[keyNNP]:
            if(posiOfPronoun>posNoun):
                if(minimumDiff>(posiOfPronoun-posNoun)):
                    minimumDiff=posiOfPronoun-posNoun
                    nearKey=keyNNP
    return nearKey

In [17]:
def pronounReplaceWithNearNoun(lineIn,PRP,NNP):
    replacePRP=[]       
    for key in PRP.keys():            
        for pos in PRP[key]:
            print('---------',key,'------',pos ,'-----')
            nearNoun=getNearestPreviousNoun(NNP,pos,lineIn)
            replacePRP.append((key,pos,nearNoun))  
    
    replacePRP=sorted(replacePRP,key=lambda x:(-x[1],x[0],x[2]))
    lineInReplacePronn=lineIn
    for prpRep in replacePRP:
        lineInReplacePronn=lineInReplacePronn[:prpRep[1]]+prpRep[2]+lineInReplacePronn[prpRep[1]+len(prpRep[0]):]
    return lineInReplacePronn

In [18]:
def obtainPriorotyOfALine(wtForLine):
    orderdLinesByWt=np.argsort(wtForLine)
    orderdLinesByWt=orderdLinesByWt[::-1]
    priority=[0]*len(wtForLine)
    
    for i in range(len(wtForLine)):
        priority[orderdLinesByWt[i]]=i
    
    sentWtAndPriority=[]    
    for i in range(len(wtForLine)):
        sentWtAndPriority.append((wtForLine[i],priority[i]))
    
    return sentWtAndPriority

In [19]:
def obtainSummary(lineForCalc,lineForExtract,percentageOfSummary,freqOfWords):
    wtForLine=[0]*len(lineForCalc)
    print('Calcualting wt for lines......')
    for li in range(len(lineForCalc)):
    
        wtForLn=0.0
        preproccdLn2=preprocessText(lineForCalc[li])
        wInL=preproccdLn2.split()
        for w in wInL:            
            w=preprocessText(w)
            if w in list(freqOfWords.keys()):    
                wtForLn=wtForLn+freqOfWords[w]
        if(len(wInL)>0):
            wtForLine[li]=(wtForLn/len(wInL))
        else:
            wtForLine[li]=0
        
    sentWtAndPriority=obtainPriorotyOfALine(wtForLine)
    print(sentWtAndPriority)
    numOfLinesInSummary=int((percentageOfSummary*len(lineForCalc))/100)
    reducedSummary=[]
    for li in range(len(lineForExtract)):
        if(sentWtAndPriority[li][1]<numOfLinesInSummary):    
            reducedSummary.append(lineForExtract[li])
    return reducedSummary

In [20]:
def removeWeekNmMonthNm(NNP):
    entries = ('january','february','march','april','may','june','july','august','september','october','november','december','monday','tuesday','wednesday','thursday','friday','saturday','sunday')
    delKeys=[]
    for key in NNP.keys():
        if key.lower() in entries:
            print(key)
            delKeys.append(key)
    
    for key in delKeys:
        del NNP[key]
    return NNP

In [21]:
def mainSummaryCalling(lineIn,percentageOfSummary):
    linePreProcessed=preprocessText(lineIn)
    rmdStopWordsLn= ' '.join(i for i in linePreProcessed.split() if i not in stopWords)
    nt=len(rmdStopWordsLn.split())
    freqOfWords = Counter(re.split(r'\s+',re.sub(r'[.,;\-!?]','',rmdStopWordsLn)))
    for word, freq in freqOfWords.items():          
        freqOfWords[word]=freqOfWords[word]/nt
    print(freqOfWords)
    tagged=getTagsForWords(lineIn)
    NNP=getNounPositions('NNP',tagged,lineIn)
    
    NNP=removeWeekNmMonthNm(NNP)
    PRP=getProNounPositions(tagged,lineIn)
    
    linesOriginal2=getAllLines(lineIn)
    lineInReplacePronn=pronounReplaceWithNearNoun(lineIn,PRP,NNP)
    linesReplacedPronn2=getAllLines(lineInReplacePronn)
    print(percentageOfSummary)
    
    reducedSummaryWithReplc=obtainSummary(linesReplacedPronn2,linesOriginal2,percentageOfSummary,freqOfWords)    
    return (reducedSummaryWithReplc)



In [23]:
cnt=0
percentageOfSummary=75

for filename in filenames:
    print(filename)
    data = open(path+'/'+filename, "r" , encoding="utf8")
    lineIn = data.read()
    reducedSummaryWithReplc=mainSummaryCalling(lineIn,percentageOfSummary)
    reducedSummaryWithReplcText='\n'.join(item for item in reducedSummaryWithReplc)
    print(reducedSummaryWithReplcText)
    data.close()

    

negative_0.txt
Counter({'international': 0.029787234042553193, 'need': 0.01702127659574468, 'costs': 0.01702127659574468, 'colonisation': 0.01702127659574468, 'israel’s': 0.01702127659574468, 'palestinians': 0.01276595744680851, 'document': 0.01276595744680851, 'expose': 0.01276595744680851, 'occupation': 0.01276595744680851, 'world': 0.01276595744680851, 'efforts': 0.01276595744680851, 'also': 0.01276595744680851, 'law': 0.01276595744680851, 'recently': 0.00851063829787234, 'put': 0.00851063829787234, 'better': 0.00851063829787234, 'economic': 0.00851063829787234, 'gross': 0.00851063829787234, 'israeli': 0.00851063829787234, 'human': 0.00851063829787234, 'united': 0.00851063829787234, 'nations': 0.00851063829787234, 'social': 0.00851063829787234, 'colonialism': 0.00851063829787234, 'critical': 0.00851063829787234, 'community': 0.00851063829787234, 'displacement': 0.00851063829787234, 'replacement': 0.00851063829787234, 'palestine': 0.00851063829787234, 'violate': 0.00851063829787234, 

key= US
key= Afghanistan
key= India
key= Pakistan
key= Basel
key= AML
key= Index
key= George
key= Monbiot
key= ‘
key= Guardian
key= ’
key= Jason
key= Hickel
key= Divide
key= Global
key= Financial
key= Integrity
key= Money
key= FATF
key= Transparency
key= International
key= G7
key= Ronojit
key= Banerjee
key= EU
key= Bank
key= Credit
key= Commerce
key= BCCI
key= Oliver
key= Bullough
key= Britain
key= UK
key= Companies
key= House
key= First
key= Pakistani
key= Second
key= Senator
key= Elizabeth
key= Warren
key= America
key= Extradition
key= Act
key= UAE
key= Pakistanis
key= NAB
key= Protection
key= Economic
key= Reforms
key= PERA
key= Foreign
key= Private
key= Investment
key= Section
key= Income
key= Tax
key= Ordinance
key= James
key= Crabtree
key= Billionaire
key= Raj
key= Switzerland
key= Accountability
key= Northern
key= Iowa
--------- them ------ 676 -----
--------- them ------ 1545 -----
--------- they ------ 1417 -----
--------- they ------ 1577 -----
--------- they ------ 1969 ----

key= Organization
key= Motor
key= Vehicle
key= Manufacturers
key= OICA
key= Karachi
key= Lahore
key= Islamabad
key= Peshawar
key= Development
key= Authority
key= LDA
key= Again
key= Uber
key= Careem
--------- it ------ 442 -----
--------- it ------ 1048 -----
--------- it ------ 1960 -----
--------- it ------ 3148 -----
--------- it ------ 3581 -----
--------- it ------ 3922 -----
--------- it ------ 4150 -----
--------- it ------ 4738 -----
--------- you ------ 920 -----
--------- you ------ 933 -----
--------- we ------ 1872 -----
--------- we ------ 2104 -----
--------- we ------ 2759 -----
--------- they ------ 2403 -----
--------- they ------ 2445 -----
--------- they ------ 3849 -----
--------- he ------ 4116 -----
--------- she ------ 4122 -----
75
Calcualting wt for lines......
[(0.004775712092785264, 2), (0.00360310421286031, 8), (0.0014254038644282545, 61), (0.0013644891693672181, 63), (0.0013858093126385808, 62), (0.0022172949002217295, 33), (0.0038802660753880264, 7), (0.00

key= ’
key= Benyamin
key= Netanyahu
key= Bibi
key= Iran
key= Israel
key= Middle
key= East
key= United
key= Kamikaze
key= Tel
key= Aviv
key= Netanya
key= Haifa
key= Shalom
key= Has
key= Cuban
key= First
key= Washington
key= Tehran
key= Isis
key= Iraq
key= US
key= Congress
key= Syria
key= Lebanon
key= Hezbollah
key= Saudi
key= Arabia
key= Egypt
key= Arab
key= Emirates
key= Secretary
key= State
key= John
key= Kerry
key= Foreign
key= Minister
key= Javad
key= Zarif
key= Sunni
key= House
key= Saud
key= Sisi
key= Imagine
key= Poor
key= Arabs
key= God
key= Anwar
key= Sadat
key= Hafez
key= Does
key= Hasan
key= Nasrallah
key= Israeli
key= Golan
key= Heights
key= Jihad
key= Mughniyeh
key= General
key= Allah
key= Dadi
key= President
key= Bashar
key= Stalin
key= Russia
key= Mao
key= China
key= Kim
key= North
key= Korea
key= Pakistan
key= Well
key= Quite
key= Riyadh
key= Apart
--------- himself ------ 335 -----
--------- he ------ 499 -----
--------- we ------ 603 -----
--------- we ------ 5261 ----

[(0.006144393241167435, 5), (0.0015360983102918587, 74), (0.003456221198156682, 22), (0.0007680491551459293, 93), (0.0064516129032258064, 4), (0.00397989107666527, 17), (0.001152073732718894, 90), (0.0021395655036208034, 51), (0.001974983541803818, 56), (0.0012568077084206116, 79), (0.005472350230414746, 6), (0.00389932647997164, 18), (0.0035842293906810036, 20), (0.0016897081413210445, 65), (0.001152073732718894, 84), (0.001075268817204301, 91), (0.0018975332068311196, 60), (0.0019201228878648233, 57), (0.006528417818740399, 3), (0.001152073732718894, 81), (0.002304147465437788, 40), (0.001728110599078341, 63), (0.002592165898617511, 33), (0.00219941348973607, 49), (0.0015360983102918587, 71), (0.001152073732718894, 87), (0.0, 99), (0.001152073732718894, 83), (0.0027649769585253456, 31), (0.0021395655036208034, 50), (0.0, 98), (0.0037066720096173107, 19), (0.0031682027649769587, 26), (0.005027230833682447, 9), (0.0016129032258064516, 69), (0.0033794162826420887, 23), (0.00230414746543

...

key= Amid
key= Indus
key= River
key= System
key= Authority
key= IRSA
key= Thursday
key= Rabi
key= Federal
key= Committee
key= Agriculture
key= FCA
key= Minister
key= Food
key= Security
key= Research
key= Syed
key= Fakhar
key= Imam
key= Punjab
key= Seed
key= Corporation
key= Kharif
key= Pakistan
key= April
key= September
key= October
key= March
key= Sindh
key= Khyber
key= Pakhtunkhwa
key= Balochistan
Thursday
April
September
October
March
--------- he ------ 772 -----
--------- he ------ 1060 -----
--------- he ------ 1309 -----
--------- we ------ 1256 -----
75
Calcualting wt for lines......
[(0.0034482758620689655, 49), (0.01, 17), (0.01111111111111111, 15), (0.003793103448275862, 46), (0.006896551724137931, 27), (0.0022988505747126436, 57), (0.005172413793103448, 37), (0.002988505747126437, 53), (0.014942528735632182, 11), (0.018620689655172412, 6), (0.01293103448275862, 13), (0.005172413793103448, 39), (0.012413793103448275, 14), (0.012931034482758622, 12), (0.027586206896551724, 1)

positive_31.txt
Counter({'production': 0.03684210526315789, 'said': 0.031578947368421054, 'auto': 0.015789473684210527, 'sector': 0.015789473684210527, 'demand': 0.015789473684210527, 'covid19': 0.015789473684210527, '”': 0.015789473684210527, 'jamali': 0.015789473684210527, 'lockdown': 0.015789473684210527, 'line': 0.010526315789473684, 'picking': 0.010526315789473684, 'company': 0.010526315789473684, 'provided': 0.010526315789473684, 'ceo': 0.010526315789473684, 'imc': 0.010526315789473684, 'single': 0.010526315789473684, 'however': 0.010526315789473684, 'commitment': 0.010526315789473684, 'doubleshift': 0.010526315789473684, 'meet': 0.010526315789473684, 'delivery': 0.010526315789473684, '4': 0.010526315789473684, 'units': 0.010526315789473684, 'islamabad': 0.005263157894736842, 'set': 0.005263157894736842, 'bounce': 0.005263157894736842, 'back': 0.005263157894736842, 'indus': 0.005263157894736842, 'motor': 0.005263157894736842, '(imc)': 0.005263157894736842, 'resumed': 0.0052631578

key= President
key= Macron
key= West
key= Muslim
key= Austria
key= Saudi
key= Arabia
key= Pakistan
key= Islamophobia
key= Muslims
key= Islam
key= Ottoman
key= Empire
key= Israel
key= ’
key= Washington
key= London
key= USSR
key= Bosnia
key= Chechnya
key= Kashmiris
key= India
key= UK
key= France
key= Germany
key= Gallup
key= Fifty-two
key= Canadians
key= US
key= Islam.
key= ”
key= Quran
key= American
key= Trump
key= Prophet
key= Center
key= Strategic
key= International
key= Studies
key= CSIS
key= Islamic
key= Middle
key= East
key= North
key= Africa
key= ISIS
key= Afghanistan
key= Iraq
key= Syria
key= Europe
key= Robert
key= Pape
key= Win
key= University
key= Maryland
key= Shariah
--------- them ------ 2199 -----
--------- them ------ 2668 -----
--------- them ------ 3979 -----
--------- them ------ 4033 -----
--------- it ------ 2812 -----
--------- it ------ 3763 -----
--------- it ------ 5090 -----
--------- they ------ 2980 -----
75
Calcualting wt for lines......
[(0.00391628145865434

key= Anwar
key= Ahmed
key= Zai
key= Shah
Saturday
May
September
--------- he ------ 466 -----
--------- he ------ 497 -----
--------- he ------ 506 -----
--------- he ------ 906 -----
--------- he ------ 1299 -----
--------- he ------ 1338 -----
--------- he ------ 1467 -----
--------- he ------ 1511 -----
--------- he ------ 1545 -----
--------- he ------ 1766 -----
--------- he ------ 1858 -----
--------- he ------ 2061 -----
--------- he ------ 2189 -----
--------- he ------ 2398 -----
--------- he ------ 2551 -----
--------- he ------ 2628 -----
--------- he ------ 2684 -----
--------- he ------ 2777 -----
--------- he ------ 3311 -----
--------- he ------ 3335 -----
--------- her ------ 811 -----
--------- her ------ 819 -----
--------- she ------ 1008 -----
--------- she ------ 1017 -----
--------- she ------ 1031 -----
--------- they ------ 1143 -----
--------- they ------ 2720 -----
--------- they ------ 2977 -----
--------- them ------ 1999 -----
--------- him ------ 2495 ----

key= UBL
key= United
key= Bank
key= Ltd
key= Literary
key= Awards
key= Saturday
key= Pakistani
key= Dr
key= Asghar
key= Nadeem
key= Syed
key= Asif
key= Aslam
key= Farrukhi
key= Ghazi
key= Salahuddin
key= Arifa
key= Syeda
key= Kishwar
key= Naheed
key= Anwaar
key= Ahmed
key= Masood
key= Ashar
key= Urdu
key= Akhtar
key= Raza
key= Salimi
key= Jindar
key= ’
key= Nasir
key= Abbas
key= Nayyar
key= Us
key= Munawwar
key= Husain
key= Dur
key= Shanul
key= Haq
key= Haqqi
key= Nazmein
key= Amina
key= Azfar
key= Partition
key= Author
key= Zehra
key= Nigah
key= Lifetime
key= Achievement
key= Award
key= Tariq
key= Khosa
key= Faltering
key= State
key= Fahmida
key= Riaz
key= Faizan
key= President
key= CEO
key= Sima
key= Kamil
key= ”
key= Pakistan
key= Salman
key= Alvi
key= Faiz
key= Ai
key= ‘
key= Hum
Saturday
--------- it ------ 1435 -----
--------- we ------ 1517 -----
--------- we ------ 1610 -----
--------- she ------ 1689 -----
--------- us ------ 707 -----
--------- us ------ 1951 -----
75
Calcual